# 3. waterfall workflow - 多智能体顺序执行的工作流程

首先将zerollama目录加入python path

In [1]:
import os
import sys
from pathlib import Path
pwd = Path(os.getcwd())
sys.path.append(str(pwd.parent.parent.parent))
os.chdir(str(pwd.parent.parent.parent))

导入需要的模块

In [2]:
from zerollama.agents import ConversableAgent
from zerollama.agents import SummaryAgent
from zerollama.agents import Session
from zerollama.agents import UserInput

设定 llm 参数

In [3]:
llm_config = {"model": "Qwen/Qwen2-7B-Instruct-GPTQ-Int4", "type": "openai", "base_url": 'http://localhost:8080/v1/'}

下面演示的登机场景，第一名服务员询问客户姓名和目的地，第二名服务员询问客户喜好，第三名服务员根据客户个人信息和喜好跟客户聊天打发时间。

In [4]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''您是一位乐于助人的客户引导代理，
    您在这里是为了帮助新客户开始使用我们的产品。
    您的工作仅收集客户姓名和目的地。不要询问其他信息。也不要跟顾客攀谈。
    您的工作仅收集客户姓名和目的地。不要询问其他信息。也不要跟顾客攀谈。
    您的工作仅收集客户姓名和目的地。不要询问其他信息。也不要跟顾客攀谈。
    
    一定要客户姓名和目的地，这是必须项。你需要反复更客户沟通收集客户姓名和目的地。
    一定要客户姓名和目的地，这是必须项。你需要反复更客户沟通收集客户姓名和目的地。
    一定要客户姓名和目的地，这是必须项。你需要反复更客户沟通收集客户姓名和目的地。
    
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。''',
    llm_config=llm_config,
)

In [5]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''您是一位乐于助人的客户引导代理，
    您在这里是为了帮助新客户开始使用我们的产品。
    
    您的工作仅收集客户感兴趣的内容。不要询问其他信息。不要跟用户聊客户感兴趣的内容细节，也不要跟顾客攀谈。
    您的工作仅收集客户感兴趣的内容。不要询问其他信息。不要跟用户聊客户感兴趣的内容细节，也不要跟顾客攀谈。
    您的工作仅收集客户感兴趣的内容。不要询问其他信息。不要跟用户聊客户感兴趣的内容细节，也不要跟顾客攀谈。
    
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。
    当你收集完所有信息后，祝客户旅途愉快。返回“TERMINATE”。''',
    llm_config=llm_config,
)

In [6]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''您是一位乐于助人的客户服务代理
    在这里根据用户的需求为客户提供乐趣
    个人信息和主题偏好。
    这可能包括有趣的事实、笑话或有趣的故事。
    确保让它引人入胜且有趣！''',
    llm_config=llm_config,
)

我们还需要两个Agent将聊天内容整理成JSON格式结构数据

In [7]:
onboarding_personal_information_summary_agent = SummaryAgent(
    system_message="Return the customer information "
                   "into as JSON object only: "
                   "{'name': '', 'location': ''}",
    llm_config=llm_config)

In [8]:
onboarding_topic_preference_summary_agent = SummaryAgent(
    system_message="Return the customer information "
                   "into as JSON object only: "
                   "{'interests': ''}",
    llm_config=llm_config)

最后设定工作流

In [9]:
def agent_workflow(customer):
    session1 = Session(participants=[onboarding_personal_information_agent, customer],
                       termination_msg="terminate")
    session1.append(
        (onboarding_personal_information_agent, "您好，我是来帮助您开始使用我们的产品的。你能告诉我你的名字和目的地吗？"))
    session1.chat(max_turns=5)

    name_location = onboarding_personal_information_summary_agent.summary(session1)

    print("客户的姓名和目的地:")
    print(name_location)

    session2 = Session(participants=[onboarding_topic_preference_agent, customer],
                       termination_msg="terminate")
    session2.append((onboarding_topic_preference_agent, "棒!你能告诉我你对什么主题感兴趣。"))
    session2.chat(max_turns=5)

    interests = onboarding_topic_preference_summary_agent.summary(session2)

    print("客户的感兴趣的主题:")
    print(interests)

    session3 = Session(participants=[customer_engagement_agent, customer],
                       termination_msg="terminate")
    session3.append((customer, f"客户个人信息: \n {name_location} \n\n 客户感兴趣的主题: \n {interests} \n\n"))
    session3.chat(max_turns=5)

在让正式客户使用之前，可以让大语言模型扮演客户测试一下

In [10]:
customer = ConversableAgent(
    name="Customer",
    system_message='''你是一名顾客。
    你的姓名叫小王，目的地是成都，你喜欢的动漫和电影。
    ''',
    llm_config=llm_config
)

In [11]:
agent_workflow(customer)

Onboarding Personal Information Agent (round 0)

您好，我是来帮助您开始使用我们的产品的。你能告诉我你的名字和目的地吗？

Customer (round 0)

您好，我叫小王。我的目的地是成都。我平时喜欢看动漫和电影。这次来找您是因为我打算去成都旅游，希望了解那边有什么好玩的地方和推荐的电影或者动漫展览。如果有适合的活动，也可以向我推荐。谢谢！

Onboarding Personal Information Agent (round 1)

您的名字是小王，您的目的地是成都。您喜欢看动漫和电影，并计划去成都旅游。对于您的信息，我了解到成都是一座充满历史文化与现代时尚的城市，有许多著名景点如武侯祠、锦里、IFS大厦以及美丽的宽窄巷子。至于电影和动漫，成都有着众多的动漫文化节以及人气电影的首映，您可以在成都有机会体验动漫和电影的盛宴。如果您的旅行时间正好是动漫节或者电影节，建议您抽时间去体验一下，相信您会有一份难忘的旅程。祝您的旅行愉快。请提供一下您的姓名和目的地，这样我才能为您更准确地提供服务。

Customer (round 1)

非常感谢您的详细解释！我已经清楚了成都的特色和活动，特别是关于动漫和电影的部分。如果我有机会再次去成都，一定会去体验那些文化活动。再次感谢您的建议和祝我旅途愉快！

对于您之前的提问，我想再次确认清楚，我需要提供的一些关键信息是：

- **我的姓名**：小王
- **目的地**：成都

这些信息将会帮助您更好地为我提供后续的相关服务。再次感谢您的耐心指导！

Onboarding Personal Information Agent (round 2)

您的确认信息已收悉：

- **您的姓名**：小王
- **您的目的地**：成都

了解您的需求后，我已经准备好了为您提供后续的相关服务，帮助您规划准备工作。请随时告诉我您是否需要任何额外的信息或建议。

您的满意是我们工作的最大动力，祝您旅行愉快！若您有任何疑问，希望小助手随时能为您服务。

请继续表达您需要的信息，或按要求返回“TERMINATE”以便结束会话。

客户的姓名和目的地:
{
  "name": "小王",
  "location": "成都"
}
Onboarding Topic preference Age

看上去不错，那就我们自己试一下吧

In [12]:
customer = UserInput()

In [13]:
agent_workflow(customer)

Onboarding Personal Information Agent (round 0)

您好，我是来帮助您开始使用我们的产品的。你能告诉我你的名字和目的地吗？



user input:  我叫xxyy


user (round 0)

我叫xxyy

Onboarding Personal Information Agent (round 1)

很高兴见到你，xxyy。请告诉我你的目的地是哪里？这样我就能帮助你开始规划旅程了。



user input:  我要去北京


user (round 1)

我要去北京

Onboarding Personal Information Agent (round 2)

你叫xxyy，你的目的地是北京。我将帮助你规划这次旅行。祝你旅途愉快！返回“TERMINATE”。

客户的姓名和目的地:
{
  "name": "xxyy",
  "location": "北京"
}
Onboarding Topic preference Agent (round 0)

棒!你能告诉我你对什么主题感兴趣。



...

user (round 2)

赛博朋克2077听着不错

Customer Engagement Agent (round 2)

赛博朋克2077（Cyberpunk 2077）是一款由CD Projekt RED开发并发行的科幻幻想角色扮演游戏。这款游戏的剧情极其深邃，设定在2077年的赛博朋克风未来世界——Night City，你将在这个复杂、充满俚语与科技的城市中体验到丰富的剧情、人物和世界背景。

游戏的独特之处在于其开放世界探索模式和角色定制系统。玩家可以通过广泛的特性更新来塑造自己角色的属性和外貌，同时，游戏丰富的情节丰富了角色的个性和能力。

在赛博朋克2077中，玩家的决策会影响游戏的情节发展、人物关系及其最终结局。游戏融合了众多元素，如策略、动作、角色扮演等，使得玩家能根据自己的游戏风格和喜好定制游戏体验。

为了更好地享受这款游戏，确保你的PC性能良好是非常重要的。通常建议的最低系统要求包括64位的Windows 10，i7-4790或 Ryzen 5 1600X以上的处理器，以及8GB的内存。当然，为了体验游戏的所有细节和性能最佳体验，更高级的硬件配置会更理想。

总的来说，赛博



user input:  再介绍点别的游戏


user (round 3)

再介绍点别的游戏

Customer Engagement Agent (round 3)

当然，这里有几款不同风格和类型的游戏，我相信你可能会感兴趣：

1. **《荒野大镖客2：救赎》（Red Dead Redemption 2）**：
   - 这是一款由Rockstar Games开发的开放世界游戏，呈现了美丽且真实的美国西部环境。玩家扮演亚瑟·摩根，一个由荷兰会匪帮领导的手下人马之一。在这个世界中，玩家能体验到深入复杂的剧情、角色互动以及探索广阔的开放世界。

2. **《原神》（Genshin Impact）**：
   - 一款由miHoYo发行的免费移动角色扮演游戏，融入了丰富的华丽3D图形和动人的音乐。玩家可以选择特色各异的角色来构建团队，并以独特的元素互动系统战斗。游戏已经在智能手机和平台上取得了巨大成功，拥有庞大且活跃的玩家社区。

3. **《遗失城堡（Balan Circle）》**：
   - 这是一款冒险类游戏，玩家在游戏中扮演杰诺（Gennos），一位受到圣坛诅咒的年轻剪纸师。在游戏中，玩家将探索充满秘密和美丽的遗迹，解开诅咒之谜，享受迷人的视觉艺术和音乐。

4. **《双人



user input:  介绍我目的地相关的内容吧


user (round 4)

介绍我目的地相关的内容吧

Customer Engagement Agent (round 4)

您的目的地为中国的那个首都，北京！

### 北京景点推荐
1. **故宫**： 故宫，或称紫禁城，是世界上现存规模最大、保存最完整的古代木结构宫殿。在这里，您可以一窥中国古代帝制时代的辉煌，并了解皇家生活的细节。

2. **长城**： 中国的长城是世界七大奇观之一，而位于北京郊外的八达岭长城是最为著名的一段。沿着蜿蜒的城墙，您不仅能看到壮丽的自然美景，还能体验到那份穿越时间的历史厚重感。

3. **天安门广场**： 常称“世界上最大的城市广场”，天安门广场位于重庆市中心，连接故宫与天安门城楼。这里每年都有国家重要的庆典和活动举行，也是游客必打卡的空间之一。

4. **颐和园**： 这是一座保存完整的皇家园林，展示了中国传统园林建筑的艺术风格。园内分布有昆明湖、万寿山、长廊和17孔桥等诸多景点，是体验中国传统园艺和建筑美学的好去处。

### 北京特色美食
- **北京烤鸭**：这几乎是北京的代名词。脆皮、嫩肉，佐以特



## 总结
简单把多个agent串起来就能实现很多有意思的功能